In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from tqdm import tqdm

import pyroomacoustics as pra
import itertools

import eval_tools as eval

Used the data of experiment 3, but use threshold to find the number of sources.

In [2]:
# load data
exp_df = pd.read_csv('results/experiment_results_exp-3.csv')
# laod picke file with the ground truth

with open('results/experiment_results_exp-3_with_ang_specs.pkl', 'rb') as f:
    ang_specs = pickle.load(f)
    
print(exp_df.shape)
print(len(ang_specs))

doa_grid_rad = np.deg2rad(np.arange(0, 360, 6))

(45900, 28)
5100


In [3]:
print(exp_df['seed'].unique())

[ 13  42 666]


In [4]:
ang_specs_dicts = {}
for d in tqdm(ang_specs):
    # if the frame id is not in the dict, add it
    if d['frame_id'] not in ang_specs_dicts:
        ang_specs_dicts[d['frame_id']] = {}
    # add the method id to the frame id
    ang_specs_dicts[d['frame_id']][d['method_id']] = d

100%|██████████| 5100/5100 [00:00<00:00, 1489516.77it/s]


In [5]:
def detect_peaks(values):
    # make circular
    n_points = len(values)
    val_ext = np.append(values, values[:10])

    # run peak finding
    indexes = pra.doa.detect_peaks(val_ext, show=False) % n_points
    candidates = np.unique(indexes)  # get rid of duplicates, if any
    return candidates

In [6]:
thresholds_ang_spec = [0.05, 0.1, 0.2, 0.5, 0.8]

df_results = pd.DataFrame()

for t in thresholds_ang_spec:
    for frame_id in tqdm(ang_specs_dicts.keys(), desc='Iterating over frames for thr {}'.format(t)):
        for method_id in ang_specs_dicts[frame_id].keys():
                        
            ang_spec = ang_specs_dicts[frame_id][method_id]['ang_spec']
            
            ang_spec = np.mean(np.array(ang_spec), -1)
            ang_spec = ang_spec / np.max(ang_spec)
            
            # set the threshold
            ang_spec[ang_spec < t] = 0
            
            # find peaks
            peaks = detect_peaks(ang_spec)      
            
            df_ = pd.DataFrame({
                'frame_id': frame_id,
                "method_id": method_id,
                'thresholds_ang_spec': t,
                'peaks_locations': peaks.tolist(),
                'n_peaks': len(peaks)
            })
            df_results = pd.concat([df_results, df_], ignore_index=True)
            
print(len(df_results))

Iterating over frames for thr 0.8: 100%|██████████| 150/150 [00:03<00:00, 38.15it/s]

88769


In [7]:
# merge the results_df with the exp_df on the frame_id and method_id
df_merge = df_results.merge(exp_df, on=['frame_id', 'method_id'])
print(df_merge['seed'].unique())


[ 13  42 666]


In [ ]:
# for each frame id and method id, get the assorciate data_frame
frame_ids = df_results['frame_id'].unique()
method_ids = df_results['method_id'].unique()

ang_error_thr = 10

results_thr_list = []

for frame_id in tqdm(frame_ids, desc='Iterating over frames'):
    for method_id in method_ids:
        df_ = df_merge.loc[
            (df_merge['frame_id'] == frame_id) 
            & (df_merge['method_id'] == method_id)
        ]
        
        thrs = df_['thresholds_ang_spec'].unique()
        
        for thr in thrs:
            
            n_sources = df_.loc[df_['thresholds_ang_spec'] == thr, 'n_sources'].unique()
            assert len(n_sources) == 1
            n_sources = n_sources[0]
            estimated_peaks = df_.loc[df_['thresholds_ang_spec'] == thr, 'peaks_locations']
            estimated_azimuths = np.rad2deg(doa_grid_rad[estimated_peaks])
            target_azimuths = np.rad2deg(df_.loc[df_['thresholds_ang_spec'] == thr, 'doas_ref_az'].unique())
        
            n_estimated = len(estimated_azimuths)
            n_target = len(target_azimuths)

            metrics = eval.compute_metrics(estimated_azimuths, target_azimuths, ang_error_thr)
            if len(target_azimuths) > n_sources:
                print(frame_id)
                print(estimated_azimuths)
                print(target_azimuths)
                print(n_sources)
                print(method_id)
                print(df_.loc[df_['thresholds_ang_spec'] == thr])
                raise ValueError('True positives greater than n_sources')
            metrics['frame_id'] = frame_id
            metrics['method_id'] = method_id
            metrics['ang_error_thr'] = ang_error_thr
            
            results_thr_list.append(metrics)
        
df_results_thr_ = pd.DataFrame(results_thr_list)
df_results_thr = df_results_thr_.merge(df_merge, on=['frame_id', 'method_id'])

Iterating over frames:   0%|          | 0/150 [00:00<?, ?it/s]

nSrc-1_doas-[44]_type-speech-duration-0.5-snr-20_noise-awgn_reverb-False_mc-0
[108. 270.]
[264. 264.]
1
alpha-2.0_beta-2_eps-1E-3_iter-500_freqs-[200, 4000]_ref_nObs-8_seed-13_norm-True
                                            frame_id  \
0  nSrc-1_doas-[44]_type-speech-duration-0.5-snr-...   
1  nSrc-1_doas-[44]_type-speech-duration-0.5-snr-...   

                                           method_id  thresholds_ang_spec  \
0  alpha-2.0_beta-2_eps-1E-3_iter-500_freqs-[200,...                 0.05   
1  alpha-2.0_beta-2_eps-1E-3_iter-500_freqs-[200,...                 0.05   

   peaks_locations  n_peaks  \
0               18        2   
1               45        2   

                                            exp_name             time  \
0  exp-3_nSrc-1_doas-[44]_type-speech-duration-0....  20250306-123716   
1  exp-3_nSrc-1_doas-[44]_type-speech-duration-0....  20250306-123716   

  record_id  num_srcs  src_ids  ...  noise_type  add_reverberation  mc_seed  \
0        s0         

ValueError: True positives greater than n_sources

In [15]:
df_results_thr['seed'].unique()

array([ 13,  42, 666])

In [16]:
# keep only 
df_results_thr['loc_method_simple'] = df_results_thr['loc_method'].apply(lambda x: x.split('-')[0])

# select the data frame with the desired parameters
df_ = df_results_thr.loc[
    (df_results_thr['n_sources'] == 3)
    & (df_results_thr['seed'] == 13)
    & (df_results_thr['record_id'] == 's0')
    & (df_results_thr['duration'] == 0.5)
    & (df_results_thr['loc_method_simple'].isin(['srp_phat', 'music', 'alpha']))
]
len(df_)

18390

In [17]:
df_.columns

Index(['true_positives', 'false_positives', 'false_negatives', 'precision',
       'recall', 'f1_score', 'accuracy', 'mean_error', 'frame_id', 'method_id',
       'ang_error_thr', 'thresholds_ang_spec', 'peaks_locations', 'n_peaks',
       'exp_name', 'time', 'record_id', 'num_srcs', 'src_ids', 'doas_est_idx',
       'doas_ref_idx', 'doas_ref_az', 'doas_est_az', 'doas_ref_el',
       'doas_est_el', 'errors', 'target_doa', 'n_sources', 'duration', 'snr',
       'noise_type', 'add_reverberation', 'mc_seed', 'loc_method', 'freq_min',
       'freq_max', 'sv_method', 'nObs', 'seed', 'sv_normalization',
       'loc_method_simple'],
      dtype='object')

In [ ]:
frame_ids = df_['frame_id'].unique()
method_ids = df_['method_id'].unique()

for frame_id in tqdm(frame_ids, desc='Iterating over frames'):
    
    for method_id in method_ids:
        
        df__ = df_.loc[
            (df_['frame_id'] == frame_id) 
            & (df_['method_id'] == method_id)
        ]
                
        if len(df__) == 0:
            continue
        
        thrs = df__['thresholds_ang_spec'].unique()

        for thr in thrs:
            precision = df__.loc[df__['thresholds_ang_spec'] == thr, 'precision'].values
            recall = df__.loc[df__['thresholds_ang_spec'] == thr, 'recall'].values
            
            
            print(precision)    
            
            1/0

Iterating over frames:   0%|          | 0/15 [00:00<?, ?it/s]

[0.5        0.5        0.5        0.5        0.5        0.5
 0.5        0.5        0.66666667 0.66666667 0.66666667 0.66666667
 0.5        0.5        0.5        0.5        0.33333333 0.33333333
 0.33333333 0.33333333]


ZeroDivisionError: division by zero

In [38]:
print(thr)
df___ = df__.loc[df__['thresholds_ang_spec'] == thr]
for column in df___.columns:
    print(column, df___[column].unique())

0.05
true_positives [6 3 1]
false_positives [6 3 2]
false_negatives [6 3 2]
precision [0.5        0.66666667 0.33333333]
recall [0.5        0.66666667 0.33333333]
f1_score [0.5        0.66666667 0.33333333]
accuracy [0.33333333 0.5        0.2       ]
mean_error [3. 2. 6.]
frame_id ['nSrc-3_doas-[44 47 53]_type-speech-duration-0.5-snr-20_noise-awgn_reverb-False_mc-0']
method_id ['alpha-2.0_beta-2_eps-1E-3_iter-500_freqs-[200, 4000]_ref_nObs-8_seed-13_norm-True']
ang_error_thr [10]
thresholds_ang_spec [0.05]
peaks_locations [18 41 46 53]
n_peaks [4]
exp_name ['exp-3_nSrc-3_doas-[44 47 53]_type-speech-duration-0.5-snr-20_noise-awgn_reverb-False_mc-0_alpha-2.0_beta-2_eps-1E-3_iter-500_freqs-[200, 4000]_ref_nObs-8_seed-13_norm-True']
time ['20250306-124246']
record_id ['s0']
num_srcs [3]
src_ids [0]
doas_est_idx [41]
doas_ref_idx [44]
doas_ref_az [4.60766923]
doas_est_az [4.29350996]
doas_ref_el [1.57079633]
doas_est_el [1.57079633]
errors [0.31415927]
target_doa ['[1.5707963267948966, 4.60

In [42]:
df___.to_csv('df_results_thr.csv', index=False)